In [ ]:
!pip install -U bitsandbytes transformers qwen_vl_utils -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.5 MB/s eta 0:00:00


In [2]:
!pip install decord

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 106.2 MB/s eta 0:00:00


In [2]:
!pip install --upgrade accelerate diffusers transformers optimum[openvino]

  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.0
    Uninstalling accelerate-0.30.0:
      Successfully uninstalled accelerate-0.30.0


# Phi mini Vision

In [5]:
!pip install

ERROR: You must give at least one requirement to install (see "pip help install")


In [4]:
!pip install  requests==2.31.0 transformers==4.43.0 accelerate==0.30.0 #torch==2.3.0 torchvision==0.18.0 # flash_attn==2.5.8 Pillow==10.3.0 --no-build-isolation

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.31.0 which is incompatible.


In [6]:
!pip install numpy==2.2.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 105.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.


In [ ]:
!pip install transformers==4.51.3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum-intel 1.22.0 requires transformers<4.49,>=4.36, but you have transformers 4.51.3 which is incompatible.


In [1]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import time

# --- Configuration ---
MODEL_ID = "unsloth/Qwen2.5-VL-7B-Instruct-unsloth-bnb-4bit"
# Or, if you want to try the larger model (ensure you have the resources and uncomment Flash Attention 2)
# MODEL_ID = "Qwen/Qwen2.5-VL-7B-Instruct"

# --- Model and Processor Loading ---
# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype="auto", # or torch.bfloat16 for the 7B model
    device_map="auto",
    # attn_implementation="flash_attention_2", # Uncomment for 7B model if supported
)

# default processor
processor = AutoProcessor.from_pretrained(MODEL_ID)

IMAGE_PATH = "/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg"
# IMPORTANT: Make sure this image path is correct and accessible.
# If running in Colab, upload the image to /content/ or adjust the path.

# processor = AutoProcessor.from_pretrained(MODEL_ID)

# --- System Prompt for Full Text Extraction ---
# system_prompt_content = """You are an expert Optical Character Recognition (OCR) assistant.
# Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image.
# Transcribe the text exactly as it appears on the document, character by character.
# You MUST ONLY output the extracted text. Do NOT add any introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not directly read from the image.
# Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.
# """

system_prompt_content = """Extract ALL text, numbers, in all sides. You MUST ONLY output the extracted text. Do NOT add any introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not directly read from the image.
Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.
"""

# --- Message Construction ---
messages = [
    {
        "role": "system",
        "content": system_prompt_content
    },
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": IMAGE_PATH,
            },
            {"type": "text", "text": "Perform a complete and verbatim Optical Character Recognition (OCR) of this entire document image. Extract every single word, number, symbol, and punctuation mark. Maintain the original layout, including line breaks and spacing, as accurately as possible."},
        ],
    }
]

print("Starting text generation...")
start_time = time.perf_counter()

# --- Preparation for Inference ---
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

# --- Inference: Generation of the output ---
# Increased max_new_tokens slightly, can be increased further if text is very dense
generated_ids = model.generate(
    **inputs,
    max_new_tokens=4096,
    do_sample=False,     # Crucial for OCR: ensures deterministic, greedy decoding
    temperature=0.01,     # With do_sample=False, this has less effect but reinforces greedy behavior
    top_k=1,             # Also reinforces greedy decoding when do_sample=False
    # repetition_penalty=1.0 # Default is 1.0 (no penalty). Adjust if repetitions are an issue.
    # eos_token_id=processor.tokenizer.eos_token_id, # Explicitly set End Of Sequence token
    # pad_token_id=processor.tokenizer.pad_token_id if processor.tokenizer.pad_token_id is not None else processor.tokenizer.eos_token_id
)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text_raw = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=True
)[0]

end_time = time.perf_counter()
elapsed_time = end_time - start_time

print("\n--- Raw Model Output (Extracted Text) ---")
print(output_text_raw)
print("-----------------------------------------")
print(f"\nTime taken: {elapsed_time:.4f} seconds")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Starting text generation...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(



--- Raw Model Output (Extracted Text) ---
CALIFORNIA USA DRIVER LICENSE
DL I1234568
EXP 08/31/2018
CLASS C
LN CARDHOLDER
FN IMA
END NONE
2570 24TH STREET
SACRAMENTO, CA 95818
DOB 08/31/1977
RSTR NONE
USA Template PSD
DONOR
SEX M
HAIR BRN
HGT 5'-05"
WGT 125 lb
EYES BRN
DD 09/30/201060221/21FD/18
ISS 09/30/2010
0831977
-----------------------------------------

Time taken: 29.4200 seconds


In [1]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info # Assuming this utility is available
import time

# --- Configuration ---
MODEL_ID = "Qwen/Qwen2.5-VL-7B-Instruct"
# Or, if you want to try the larger model (ensure you have the resources and uncomment Flash Attention 2)
# MODEL_ID = "Qwen/Qwen2.5-VL-32B-Instruct" # Example if a larger model was available and you wanted to try

# --- Quantization Configuration ---
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # "nf4" (NormalFloat4) is a common choice
    bnb_4bit_compute_dtype=torch.bfloat16, # Computation dtype after dequantization
    bnb_4bit_use_double_quant=True, # Optional: for slightly better quality
)

print(f"Loading model {MODEL_ID} with 4-bit quantization...")
# --- Model and Processor Loading ---
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=quantization_config, # Apply 4-bit quantization
    device_map="auto", # Crucial for bitsandbytes with multiple GPUs or offloading
    # attn_implementation="flash_attention_2", # Uncomment for 7B/32B model if supported and FA2 is installed
    # torch_dtype is handled by bnb_4bit_compute_dtype in BitsAndBytesConfig
)
print("Model loaded.")

# default processor
processor = AutoProcessor.from_pretrained(MODEL_ID)
print("Processor loaded.")

IMAGE_PATH = "/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg"
# IMPORTANT: Make sure this image path is correct and accessible.
# If running in Colab, upload the image to /content/ or adjust the path.
# You might need to upload your image to the Colab environment if you haven't.
# For example, using the file upload feature in Colab's sidebar.

# --- System Prompt for Full Text Extraction ---
system_prompt_content = """You are an expert Optical Character Recognition (OCR) assistant.
Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image.
Transcribe the text exactly as it appears on the document, character by character.
You MUST ONLY output the extracted text. Do NOT add any introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not directly read from the image.
Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.
"""

# --- Message Construction ---
messages = [
    {
        "role": "system",
        "content": system_prompt_content
    },
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": IMAGE_PATH,
            },
            {"type": "text", "text": "Perform a complete and verbatim Optical Character Recognition (OCR) of this entire document image. Extract every single word, number, symbol, and punctuation mark. Maintain the original layout, including line breaks and spacing, as accurately as possible."},
        ],
    }
]

print("Starting text generation...")
start_time = time.perf_counter()

# --- Preparation for Inference ---
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

# Check if image path exists before processing
import os
if not os.path.exists(IMAGE_PATH):
    print(f"ERROR: Image not found at {IMAGE_PATH}. Please upload it or check the path.")
    exit()

image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device) # Ensure inputs are on the same device as the model parts

# --- Inference: Generation of the output ---
# Increased max_new_tokens slightly, can be increased further if text is very dense
generated_ids = model.generate(
    **inputs,
    max_new_tokens=4096,
    do_sample=False,     # Crucial for OCR: ensures deterministic, greedy decoding
    temperature=0.01,     # With do_sample=False, this has less effect but reinforces greedy behavior
    top_k=1,             # Also reinforces greedy decoding when do_sample=False
    # repetition_penalty=1.0 # Default is 1.0 (no penalty). Adjust if repetitions are an issue.
    # eos_token_id=processor.tokenizer.eos_token_id, # Explicitly set End Of Sequence token
    # pad_token_id=processor.tokenizer.pad_token_id if processor.tokenizer.pad_token_id is not None else processor.tokenizer.eos_token_id
)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text_raw = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=True
)[0]

end_time = time.perf_counter()
elapsed_time = end_time - start_time

print("\n--- Raw Model Output (Extracted Text) ---")
print(output_text_raw)
print("-----------------------------------------")
print(f"\nTime taken: {elapsed_time:.4f} seconds")

# --- Clean up (optional, but good practice especially in notebooks) ---
# import gc
# del model
# del processor
# del inputs
# del generated_ids
# torch.cuda.empty_cache()
# gc.collect()
# print("Cleaned up model and tensors.")

Loading model Qwen/Qwen2.5-VL-7B-Instruct with 4-bit quantization...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Model loaded.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Processor loaded.
Starting text generation...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(



--- Raw Model Output (Extracted Text) ---
CALIFORNIA USA DRIVER LICENSE
DL I1234568
EXP 08/31/2018
CLASS C
LN CARDHOLDER
FN IMA
END NONE
2570 24TH STREET
SACRAMENTO, CA 95818
DOB 08/31/1977
RSTR NONE
USA Template PSD
DONOR
SEX M
HGT 5'-05"
WGT 125 lb
EYES BRN
ISS 09/30/2010
DD 09/30/201060221/21FD/18
0831977
-----------------------------------------

Time taken: 26.8712 seconds
